In [9]:
import keras_applications
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
import numpy as np
import boto3, os, tempfile

In [10]:
MODEL_BUCKET_NAME = 'ml-models-3521'
MODEL_KEY_NAME = 'resnet50_weights_tf_dim_ordering_tf_kernels.h5'

IMG_UPLOAD_BUCKET_NAME = 'image-uploads-3521'
IMG_KEY_NAME = 'elephant.jpg'

In [11]:
temp_dir = 'tmp'

if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

s3 = boto3.resource('s3')

In [12]:
%%time 
model_path = os.path.join(temp_dir, MODEL_KEY_NAME)
print('Downloading model...')
s3.Bucket(MODEL_BUCKET_NAME).download_file(MODEL_KEY_NAME, model_path)
print('Model downloaded.')

Model downloaded.
CPU times: user 4.17 s, sys: 6.7 s, total: 10.9 s
Wall time: 28.4 s


In [13]:
model_path

'tmp/resnet50_weights_tf_dim_ordering_tf_kernels.h5'

In [14]:
%%time
print('Loading model...')
model = ResNet50(weights=model_path)
print('Model loaded.')

Loading model...
Model loaded.
CPU times: user 30.6 s, sys: 575 ms, total: 31.2 s
Wall time: 32.8 s


In [15]:
print('Downloading image...')
tmp_file = tempfile.NamedTemporaryFile(dir=temp_dir, delete=False)
img_object = s3.Bucket(IMG_UPLOAD_BUCKET_NAME).Object(IMG_KEY_NAME)
img_object.download_fileobj(tmp_file)
tmp_file.close()
print('Image downloaded to', tmp_file.name)

Image downloaded to /home/osboxes/examples/keras-example/tmp/tmpd3atgggq


In [16]:
#model = ResNet50(weights='imagenet')

In [17]:
image_path = tmp_file.name #'elephant.jpg'

In [18]:
img = image.load_img(image_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0) 
x = preprocess_input(x)

In [19]:
print('Predicting...')
preds = model.predict(x)
print(preds.shape)

Predicting...
(1, 1000)


In [20]:
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02504458', 'African_elephant', 0.60431176), ('n01871265', 'tusker', 0.3796893), ('n02504013', 'Indian_elephant', 0.015997304)]


In [7]:
keras_applications.imagenet_utils.CLASS_INDEX

{'0': ['n01440764', 'tench'],
 '1': ['n01443537', 'goldfish'],
 '2': ['n01484850', 'great_white_shark'],
 '3': ['n01491361', 'tiger_shark'],
 '4': ['n01494475', 'hammerhead'],
 '5': ['n01496331', 'electric_ray'],
 '6': ['n01498041', 'stingray'],
 '7': ['n01514668', 'cock'],
 '8': ['n01514859', 'hen'],
 '9': ['n01518878', 'ostrich'],
 '10': ['n01530575', 'brambling'],
 '11': ['n01531178', 'goldfinch'],
 '12': ['n01532829', 'house_finch'],
 '13': ['n01534433', 'junco'],
 '14': ['n01537544', 'indigo_bunting'],
 '15': ['n01558993', 'robin'],
 '16': ['n01560419', 'bulbul'],
 '17': ['n01580077', 'jay'],
 '18': ['n01582220', 'magpie'],
 '19': ['n01592084', 'chickadee'],
 '20': ['n01601694', 'water_ouzel'],
 '21': ['n01608432', 'kite'],
 '22': ['n01614925', 'bald_eagle'],
 '23': ['n01616318', 'vulture'],
 '24': ['n01622779', 'great_grey_owl'],
 '25': ['n01629819', 'European_fire_salamander'],
 '26': ['n01630670', 'common_newt'],
 '27': ['n01631663', 'eft'],
 '28': ['n01632458', 'spotted_salama